# Initialize

In [1]:
# turn on autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
ROOT = os.path.abspath("..")          # if your notebook/ lives under project-root/notebook/
SRC  = os.path.join(ROOT, "src")
sys.path.append(SRC)

# Options Underlying

In [3]:
import pandas as pd

options_df = pd.DataFrame({
    'mult':[100,100],
    'Expiration': ['1/15/27','1/15/27'],
    'Strike':[250.0,250.0],
    'Spot':[252.29,252.29],
    'CP':['P','C'],
    'EA':['A','A'],
    'Texp':[1.2416,1.2416],
    'Vol':['27.45%','27.45%'],
    'Borrow':[-0.006,-0.0060]
})

dividends_df = pd.DataFrame({
    'Date':['11/8/25','2/8/26','5/10/26','8/9/26','11/8/26','2/8/27','5/8/27','8/8/27'],
    'Amount':[0.26,0.26,0.26,0.26,0.26,0.40,0.40,0.40]
})

str_underlying='AAPL'
pd_val_date=pd.Timestamp('10/17/2025') 

In [12]:
from listed_pricer_data_loader import get_ivol_yc, apply_ivol_yc, fetch_and_pick
from listed_pricer_qlib import price_table_with_divs_ql
from listed_pricer_utils import _to_iso_date_key

yc_results=get_ivol_yc(pd_val_date)

options_df2=apply_ivol_yc(yc_results,options_df)
options_df2

priced_ql_ir=price_table_with_divs_ql(
    options_df2, dividends_df,
    #valuation_date='10/17/2025',
    valuation_date=pd_val_date.strftime('%m/%d/%Y'),
    default_r=0.0308,                  # use your carry here; bump if needed to match combo
    init_vol=0.2755,           # optional: seed for the SimpleQuote
    t_grid=400, x_grid=200,
    want_greeks=True,
    compute_combo=True
)


# legs_df with your desired contracts:
# columns: Expiration (any parseable date), Strike (float), CP ('C'/'P')
legs_df=options_df2[['Expiration','Strike','CP']]
out = fetch_and_pick(str_underlying, trade_date=pd_val_date.strftime('%Y-%m-%d'), legs_df=legs_df, snap_to_nearest=False)
#print(out["issues"])     # any misses or snaps
#print(out["matched_df"]) # exact rows you can pass to your pric
priced = priced_ql_ir.rename(columns={"CP":"cp"})
out_relevant=out["matched_df"][['expiration','strike','cp','bid','ask','iv','openInterest','volume','delta','gamma','vega','underlying_price','optionId']]
mkt    = out_relevant.rename(columns={"expiration":"Expiration",
                                      "strike":"Strike",
                                      "cp":"cp"})

priced = priced_ql_ir.rename(columns={"CP":"cp"}).copy()
priced["Exp_key"] = priced["Expiration"].map(_to_iso_date_key)
priced["Strike"]  = priced["Strike"].astype(float)
priced["cp"]      = priced["cp"].str.upper().str[0]

# market side
mkt = out_relevant.rename(columns={"expiration":"Expiration","strike":"Strike","cp":"cp"}).copy()
mkt["Exp_key"] = mkt["Expiration"].map(_to_iso_date_key)
mkt["Strike"]  = mkt["Strike"].astype(float)
mkt["cp"]      = mkt["cp"].str.upper().str[0]

# normalize strikes to cents (or your preferred tick)
priced["Strike"] = priced["Strike"].round(2)
mkt["Strike"]    = mkt["Strike"].round(2)

# safe merge (one-to-one expected)
df = priced.merge(
    mkt[['Exp_key','Strike','cp','bid','ask','iv','openInterest','volume',
         'delta','gamma','vega','underlying_price','optionId']],
    on=['Exp_key','Strike','cp'],
    how='left',
    validate='one_to_one'
)

# optional: keep the human-readable date and drop the key
df = df.drop(columns=['Exp_key'])

df_show = df[["Expiration","Strike","Spot","cp","EA","Texp","Vol","Borrow","bid","Theo","ask"]]
df_show

,Expiration,Strike,Spot,cp,EA,Texp,Vol,Borrow,bid,Theo,ask
0,1/15/27,250.0,252.29,P,A,1.2416,27.45%,-0.006,24.6,24.854431,25.00
1,1/15/27,250.0,252.29,C,A,1.2416,27.45%,-0.006,36.7,37.275696,37.65


In [4]:
from listed_pricer_data_loader import get_ivol_yc, apply_ivol_yc, fetch_and_pick
from listed_pricer_qlib import price_table_with_divs_ql
from listed_pricer_utils import _to_iso_date_key

def run_pipeline(pd_val_date, options_df, dividends_df, underlying):
    yc=get_ivol_yc(pd_val_date)

    opts2=apply_ivol_yc(yc,options_df)

    priced_ql_ir=price_table_with_divs_ql(
        opts2, dividends_df,
        #valuation_date='10/17/2025',
        valuation_date=pd_val_date.strftime('%m/%d/%Y'),
        default_r=0.0308,                  # use your carry here; bump if needed to match combo
        init_vol=0.2755,           # optional: seed for the SimpleQuote
        t_grid=400, x_grid=200,
        want_greeks=True,
        compute_combo=True
    )


    # legs_df with your desired contracts:
    # columns: Expiration (any parseable date), Strike (float), CP ('C'/'P')
    legs_df=opts2[['Expiration','Strike','CP']]
    out = fetch_and_pick(underlying, trade_date=pd_val_date.strftime('%Y-%m-%d'), legs_df=legs_df, snap_to_nearest=False)
    issues, mkt_rows = out["issues"], out["matched_df"]
    print(f"issues: {issues}")
    priced = priced_ql_ir.rename(columns={"CP":"cp"})
    out_relevant=out["matched_df"][['expiration','strike','cp','bid','ask','iv','openInterest','volume','delta','gamma','vega','underlying_price','optionId']]
    mkt    = out_relevant.rename(columns={"expiration":"Expiration",
                                        "strike":"Strike",
                                        "cp":"cp"})

    priced = priced_ql_ir.rename(columns={"CP":"cp"}).copy()
    priced["Exp_key"] = priced["Expiration"].map(_to_iso_date_key)
    priced["Strike"]  = priced["Strike"].astype(float)
    priced["cp"]      = priced["cp"].str.upper().str[0]

    # market side
    mkt = out_relevant.rename(columns={"expiration":"Expiration","strike":"Strike","cp":"cp"}).copy()
    mkt["Exp_key"] = mkt["Expiration"].map(_to_iso_date_key)
    mkt["Strike"]  = mkt["Strike"].astype(float)
    mkt["cp"]      = mkt["cp"].str.upper().str[0]

    # normalize strikes to cents (or your preferred tick)
    priced["Strike"] = priced["Strike"].round(2)
    mkt["Strike"]    = mkt["Strike"].round(2)

    # safe merge (one-to-one expected)
    combined = priced.merge(
        mkt[['Exp_key','Strike','cp','bid','ask','iv','openInterest','volume',
            'delta','gamma','vega','underlying_price','optionId']],
        on=['Exp_key','Strike','cp'],
        how='left',
        validate='one_to_one'
    )

    # optional: keep the human-readable date and drop the key
    combined = combined.drop(columns=['Exp_key'])

    combined_subset =combined[["Expiration","Strike","Spot","cp","EA","Texp","Vol","Borrow","bid","Theo","ask"]]
    return combined_subset, combined, yc, mkt

In [6]:
combined_subset, combined, yc, mkt = run_pipeline(pd_val_date, options_df, dividends_df, "AAPL")
combined_subset

issues: []


,Expiration,Strike,Spot,cp,EA,Texp,Vol,Borrow,bid,Theo,ask
0,1/15/27,250.0,252.29,P,A,1.2416,27.45%,-0.006,24.6,24.854431,25.00
1,1/15/27,250.0,252.29,C,A,1.2416,27.45%,-0.006,36.7,37.275696,37.65


In [ ]:
priced

,mult,Expiration,Strike,Spot,cp,EA,Texp,Vol,Borrow,Rate,Theo,Delta,Gamma,Theta,Vega,Combo_C_minus_P
0,100,1/15/27,250,252.29,P,A,1.2416,27.45%,-0.006,0.034684,24.854431,-0.396546,0.005454,-8.152321,1.069343,12.421265
1,100,1/15/27,250,252.29,C,A,1.2416,27.45%,-0.006,0.034684,37.275696,0.634695,0.004935,-17.063115,1.068914,12.421265
